In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import VAR
from scipy.stats import pearsonr

# Data cleaning and preprocessing

Data must be cleaned and preprocessed. 
<br>
Further advise is needed how to do it!

In [24]:
df = pd.read_excel("Data.xlsx", sheet_name="Monthly_Python_Input", index_col=0)
#df = df.drop(columns=["EMB_US_Equity"])
df["EMB_US_Equity_log"] = np.log(df.EMB_US_Equity) - np.log(df.EMB_US_Equity.shift(1))
df = df.drop(columns=["EMB_US_Equity"]).dropna()
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
print(df.head())
print(df.tail())
print(df.shape) # (168, 12)

            EMB_US_Equity_log  Dax_Index  VIX_Index  USGG10YR_Index  \
Date                                                                  
2008-02-29          -0.008737    6748.13      26.54          3.5092   
2008-03-31          -0.003259    6534.97      25.61          3.4096   
2008-04-30           0.006999    6948.82      20.79          3.7279   
2008-05-30          -0.003345    7096.79      17.83          4.0595   
2008-06-30          -0.024545    6418.32      23.95          3.9690   

            EURUSD_BGN_Curncy  LF98OAS_Index  USYC2Y10_Index  SX5E_Index  \
Date                                                                       
2008-02-29             1.5179           7.45         189.578     3724.50   
2008-03-31             1.5788           7.81         182.115     3628.06   
2008-04-30             1.5622           6.53         146.961     3825.02   
2008-05-30             1.5554           6.22         145.713     3777.85   
2008-06-30             1.5755           7.08  